In [41]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder


In [42]:
# Reading a CSV file
columns_name = [
    'timestamp','sex','matric_gpa_%' ,'year_in_2023','faculty','gpa_%_2023','accommodation_status',
    'monthly_allowance','scholarship_bursary_2023','study_hours_week','socialising_week','drinks_night',
    'classes_missed_alcohol','modules_failed','in_relationship','parental_approval_alcohol','relationship_with_parents'
]


In [43]:
student_survey=pd.read_csv('student_survey.csv',names=columns_name,header=0)
student_survey.head()

,timestamp,sex,matric_gpa_%,year_in_2023,faculty,gpa_%_2023,accommodation_status,monthly_allowance,scholarship_bursary_2023,study_hours_week,socialising_week,drinks_night,classes_missed_alcohol,modules_failed,in_relationship,parental_approval_alcohol,relationship_with_parents
0,2024/03/07 5:12:01 pm EET,Female,76.0,2nd Year,Arts & Social Sciences,72.0,Private accommodation/ stay with family/friends,R 4001- R 5000,No,8+,Only weekends,8+,3,0,Yes,Yes,Very close
1,2024/03/07 5:12:08 pm EET,Male,89.0,2nd Year,Economic & Management Sciences,75.0,Private accommodation/ stay with family/friends,R 7001 - R 8000,"Yes (NSFAS, etc...)",8+,Only weekends,3-5,4+,0,No,Yes,Very close
2,2024/03/07 5:12:25 pm EET,Male,76.0,1st Year,AgriSciences,55.0,Private accommodation/ stay with family/friends,R 4001- R 5000,No,3-5,2,8+,3,0,No,Yes,Very close
3,2024/03/07 5:12:28 pm EET,Male,89.0,2nd Year,Engineering,84.0,Private accommodation/ stay with family/friends,R 6001 - R 7000,No,3-5,3,8+,2,0,Yes,Yes,Very close
4,2024/03/07 5:13:00 pm EET,Female,74.0,2nd Year,Arts & Social Sciences,52.0,Private accommodation/ stay with family/friends,R 4001- R 5000,No,3-5,Only weekends,5-8,1,3,No,Yes,Fair


#### processing

In [44]:
student_survey.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 406 entries, 0 to 405
Data columns (total 17 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   timestamp                  406 non-null    object 
 1   sex                        404 non-null    object 
 2   matric_gpa_%               399 non-null    float64
 3   year_in_2023               333 non-null    object 
 4   faculty                    399 non-null    object 
 5   gpa_%_2023                 320 non-null    float64
 6   accommodation_status       383 non-null    object 
 7   monthly_allowance          375 non-null    object 
 8   scholarship_bursary_2023   398 non-null    object 
 9   study_hours_week           403 non-null    object 
 10  socialising_week           404 non-null    object 
 11  drinks_night               404 non-null    object 
 12  classes_missed_alcohol     403 non-null    object 
 13  modules_failed             403 non-null    object 

In [45]:
# Outlier Handling
conti_columns = ['matric_gpa_%','gpa_%_2023']
student_survey.describe()

,matric_gpa_%,gpa_%_2023
count,399.000000,320.000000
mean,77.989724,66.268469
std,7.048618,9.147906
min,34.000000,30.000000
25%,74.000000,60.000000
50%,78.000000,65.000000
75%,83.000000,73.000000
max,99.000000,95.220000


In [46]:
# Using IQR to remove Outlier

for each in conti_columns:
    q1 = student_survey[each].quantile(0.25)
    q3 = student_survey[each].quantile(0.75)
    max_b = 1.5 * (q3 - q1) + q3 
    min_b = q1 - 1.5 * (q3 - q1)

    to_drop_index = student_survey[each][(student_survey[each] > max_b) | (student_survey[each] < min_b)].index
    student_survey.drop(to_drop_index, axis = 0, inplace = True)
    
    print(f"Dropped {to_drop_index} : {max_b} | {min_b} : for column {each}")

Dropped Index([38, 54, 73, 203, 254, 268, 335, 358], dtype='int64') : 96.5 | 60.5 : for column matric_gpa_%
Dropped Index([303], dtype='int64') : 92.5 | 40.5 : for column gpa_%_2023


In [47]:
student_survey.info()

<class 'pandas.core.frame.DataFrame'>
Index: 397 entries, 0 to 405
Data columns (total 17 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   timestamp                  397 non-null    object 
 1   sex                        395 non-null    object 
 2   matric_gpa_%               390 non-null    float64
 3   year_in_2023               325 non-null    object 
 4   faculty                    390 non-null    object 
 5   gpa_%_2023                 313 non-null    float64
 6   accommodation_status       374 non-null    object 
 7   monthly_allowance          366 non-null    object 
 8   scholarship_bursary_2023   389 non-null    object 
 9   study_hours_week           394 non-null    object 
 10  socialising_week           395 non-null    object 
 11  drinks_night               395 non-null    object 
 12  classes_missed_alcohol     394 non-null    object 
 13  modules_failed             394 non-null    object 
 14 

In [48]:
# handling null values
student_survey.isna().sum()

timestamp                     0
sex                           2
matric_gpa_%                  7
year_in_2023                 72
faculty                       7
gpa_%_2023                   84
accommodation_status         23
monthly_allowance            31
scholarship_bursary_2023      8
study_hours_week              3
socialising_week              2
drinks_night                  2
classes_missed_alcohol        3
modules_failed                3
in_relationship               3
parental_approval_alcohol     3
relationship_with_parents     3
dtype: int64

In [49]:
student_survey['year_in_2023'].value_counts(dropna=False)

year_in_2023
2nd Year        149
1st Year        124
NaN              72
3rd Year         40
4th Year          7
Postgraduate      5
Name: count, dtype: int64

In [50]:
# Lets replace NAN value for column year_in_2023 to 0th year/Highschool
student_survey['year_in_2023'].fillna(value = '0th Year', inplace = True)

C:\Users\sunil\AppData\Local\Temp\ipykernel_21628\2802395453.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  student_survey['year_in_2023'].fillna(value = '0th Year', inplace = True)


In [51]:
student_survey['year_in_2023'].value_counts(dropna=False)

year_in_2023
2nd Year        149
1st Year        124
0th Year         72
3rd Year         40
4th Year          7
Postgraduate      5
Name: count, dtype: int64

In [52]:
# Rows Null values 
excess_null_rows = student_survey[student_survey.isna().sum(axis=1) >= 5].index

In [53]:
student_survey.drop(excess_null_rows, axis = 0, inplace = True)


In [54]:
student_survey.isna().sum()

timestamp                     0
sex                           0
matric_gpa_%                  5
year_in_2023                  0
faculty                       5
gpa_%_2023                   82
accommodation_status         21
monthly_allowance            29
scholarship_bursary_2023      6
study_hours_week              1
socialising_week              0
drinks_night                  0
classes_missed_alcohol        1
modules_failed                1
in_relationship               1
parental_approval_alcohol     1
relationship_with_parents     1
dtype: int64

In [55]:
# Replace categorical values with it mode
cat_cols = [
    'faculty','scholarship_bursary_2023','study_hours_week','classes_missed_alcohol',
    'modules_failed','in_relationship','parental_approval_alcohol','relationship_with_parents'
]

for each in cat_cols:
    mode_val = student_survey[each].mode()[0]
    student_survey[each].fillna(mode_val, inplace = True)

# Creating new feature with NaN value
unknown_nan = ['monthly_allowance', 'accommodation_status']
for each in unknown_nan:
    student_survey[each].fillna('Unknown', inplace=True)

# Handling a continuous data column
## TODO: Handle this more gracefully ( with condition feature like "sex" or other )
student_survey['matric_gpa_%'].fillna(student_survey['matric_gpa_%'].mean(), inplace = True)

C:\Users\sunil\AppData\Local\Temp\ipykernel_21628\3266706445.py:9: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  student_survey[each].fillna(mode_val, inplace = True)
C:\Users\sunil\AppData\Local\Temp\ipykernel_21628\3266706445.py:14: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

F

In [56]:
student_survey.isna().sum()

timestamp                     0
sex                           0
matric_gpa_%                  0
year_in_2023                  0
faculty                       0
gpa_%_2023                   82
accommodation_status          0
monthly_allowance             0
scholarship_bursary_2023      0
study_hours_week              0
socialising_week              0
drinks_night                  0
classes_missed_alcohol        0
modules_failed                0
in_relationship               0
parental_approval_alcohol     0
relationship_with_parents     0
dtype: int64

In [57]:
# Drop null values of target
student_survey.dropna(inplace = True)

In [58]:
student_survey = student_survey.drop(['timestamp'], axis=1)

In [59]:
student_survey.info()

<class 'pandas.core.frame.DataFrame'>
Index: 313 entries, 0 to 405
Data columns (total 16 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   sex                        313 non-null    object 
 1   matric_gpa_%               313 non-null    float64
 2   year_in_2023               313 non-null    object 
 3   faculty                    313 non-null    object 
 4   gpa_%_2023                 313 non-null    float64
 5   accommodation_status       313 non-null    object 
 6   monthly_allowance          313 non-null    object 
 7   scholarship_bursary_2023   313 non-null    object 
 8   study_hours_week           313 non-null    object 
 9   socialising_week           313 non-null    object 
 10  drinks_night               313 non-null    object 
 11  classes_missed_alcohol     313 non-null    object 
 12  modules_failed             313 non-null    object 
 13  in_relationship            313 non-null    object 
 14 

In [60]:
target = student_survey['gpa_%_2023']
student_survey.drop('gpa_%_2023', axis =1, inplace=True)

In [61]:
# Data Encoding and Normalization
## One-Hot encoding
nominal_cols = ['sex', 'faculty', 'year_in_2023']

# one_hot_model = OneHotEncoder(sparse_output = False)
# encoded_onehot = one_hot_model.fit_transform(student_survey[nominal_cols])

encoded_onehot = pd.get_dummies(student_survey[nominal_cols], dtype=int)
student_survey.drop(nominal_cols, axis=1, inplace = True)

In [62]:
## Label Encoding
ordinal_cols =[
    'accommodation_status','monthly_allowance','scholarship_bursary_2023','study_hours_week','socialising_week','drinks_night',
    'classes_missed_alcohol','modules_failed','in_relationship','parental_approval_alcohol','relationship_with_parents'
]

# label_encoder = LabelEncoder()
# encoded_label = label_encoder.fit_transform(student_survey[ordinal_cols])

for each in ordinal_cols:
    student_survey[each], _ = pd.factorize(student_survey[each])

In [63]:
final_dataset = pd.concat([student_survey, encoded_onehot], axis =1)

In [64]:
final_dataset.info()

<class 'pandas.core.frame.DataFrame'>
Index: 313 entries, 0 to 405
Data columns (total 28 columns):
 #   Column                                  Non-Null Count  Dtype  
---  ------                                  --------------  -----  
 0   matric_gpa_%                            313 non-null    float64
 1   accommodation_status                    313 non-null    int64  
 2   monthly_allowance                       313 non-null    int64  
 3   scholarship_bursary_2023                313 non-null    int64  
 4   study_hours_week                        313 non-null    int64  
 5   socialising_week                        313 non-null    int64  
 6   drinks_night                            313 non-null    int64  
 7   classes_missed_alcohol                  313 non-null    int64  
 8   modules_failed                          313 non-null    int64  
 9   in_relationship                         313 non-null    int64  
 10  parental_approval_alcohol               313 non-null    int64  
 11

In [65]:
# Normalize
normalize = StandardScaler()
final_norm  = normalize.fit_transform(final_dataset)

In [66]:
target = target.reset_index(drop=True)

to_save = pd.DataFrame(final_norm, columns=final_dataset.columns)
to_save['target'] = target

In [71]:
to_save.to_csv("student_cleaned_data.csv", index=False)


In [72]:
to_save

,matric_gpa_%,accommodation_status,monthly_allowance,scholarship_bursary_2023,study_hours_week,socialising_week,drinks_night,classes_missed_alcohol,modules_failed,in_relationship,...,faculty_Law,faculty_Medicine and Health Services,faculty_Science,year_in_2023_0th Year,year_in_2023_1st Year,year_in_2023_2nd Year,year_in_2023_3rd Year,year_in_2023_4th Year,year_in_2023_Postgraduate,target
0,-0.365008,-0.395407,-1.095457,-0.360505,-1.155375,-1.205451,-1.407833,-2.507187,-0.753244,-1.148268,...,-0.172062,-0.172062,-0.399073,-0.056614,-0.762108,1.076392,-0.38278,-0.151248,-0.127412,72.0
1,1.794677,-0.395407,-0.464006,2.773886,-1.155375,-1.205451,-0.571682,-1.676764,-0.753244,0.870877,...,-0.172062,-0.172062,-0.399073,-0.056614,-0.762108,1.076392,-0.38278,-0.151248,-0.127412,75.0
2,-0.365008,-0.395407,-1.095457,-0.360505,-0.362322,-0.498885,-1.407833,-2.507187,-0.753244,0.870877,...,-0.172062,-0.172062,-0.399073,-0.056614,1.312151,-0.929029,-0.38278,-0.151248,-0.127412,55.0
3,1.794677,-0.395407,0.167446,-0.360505,-0.362322,0.207681,-1.407833,-0.846341,-0.753244,-1.148268,...,-0.172062,-0.172062,-0.399073,-0.056614,-0.762108,1.076392,-0.38278,-0.151248,-0.127412,84.0
4,-0.697268,-0.395407,-1.095457,-0.360505,-0.362322,-1.205451,0.264470,-0.015919,-0.077699,0.870877,...,-0.172062,-0.172062,-0.399073,-0.056614,-0.762108,1.076392,-0.38278,-0.151248,-0.127412,52.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
308,-0.032749,-0.395407,-1.095457,-0.360505,0.430732,-1.205451,1.100621,0.814504,-0.753244,0.870877,...,-0.172062,5.811865,-0.399073,-0.056614,-0.762108,1.076392,-0.38278,-0.151248,-0.127412,74.0
309,1.296288,-0.395407,-1.095457,-0.360505,1.223785,-0.498885,-0.571682,-0.846341,-0.753244,-1.148268,...,-0.172062,-0.172062,-0.399073,-0.056614,1.312151,-0.929029,-0.38278,-0.151248,-0.127412,70.0
310,-0.697268,-0.395407,0.167446,-0.360505,1.223785,-1.205451,1.100621,0.814504,-0.077699,0.870877,...,-0.172062,-0.172062,-0.399073,-0.056614,-0.762108,1.076392,-0.38278,-0.151248,-0.127412,60.0
311,-0.863397,-0.395407,0.167446,2.773886,1.223785,0.914247,-1.407833,0.814504,-0.753244,-1.148268,...,-0.172062,-0.172062,-0.399073,-0.056614,-0.762108,1.076392,-0.38278,-0.151248,-0.127412,58.0
